In [1]:
import numpy as np
import pandas as pd

-----------------------------------------

# 1.原始数据与变量构造  
# 2.

-----------------------------------------

# 1.原始数据与变量构造

## 1.1 原始数据

In [5]:
raw_data = pd.read_excel("GoyalWelch_PredictorData2018_monthly.xlsx", index_col=0)
raw_data.head()

,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
yyyymm,,,,,,,,,,,,,,,,,
187101,4.44,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004955,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187102,4.50,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004514,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187103,4.61,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004243,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187104,4.74,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004632,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187105,4.86,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003691,NaN,NaN,NaN,NaN,NaN,NaN,NaN


对原始数据的index和列名说明如下：  
    1. yyyy-mm：四位年份-两位月份
    2. Index：S&P 500 index
    3. D12: "12-month movingsumsofdividendspaidonthe S&P 500 index"
    4. E12: "12-month moving sums of earnings on the S&P 500 index"
    5. b/m: "the ratio of book value to market value for the Dow Jones Industrial Average"
    6. tbl: "Treasury-bill rates"
    7. AAA: "AAA-rated corporate bond yields"
    8. BAA: "BAA-rated corporate bond yields"
    9. lty: "long-term government bond yield"
    10. ntis: "*Net Equity Expansion* (ntis) is the ratio of 12-month moving sums of net issues by NYSE listed stocks divided by the total end-of-year market capitalization of NYSE stocks"
    11. Rfree: "Treasury-bill rate" and construction
    12. infl: "Consumer Price Index (All Urban Consumers)"
    
>***Because inﬂation information is released only in the following month, we wait for one month before using it in our monthly regressions.***

    13. ltr: "Long TermRate of Returns"
    14. coppr:
    15. svar: "Stock Variance is computed as sum of squared daily returns on the S&P 500"
    16. csp: "The cross-sectional beta premium measures the relative valuations of high- and low-beta stocks"

根据Welch and Goyal (2008)，需要构造12个用于预测的变量：
 1. dp: $ log(D12) - log(Index) $
 2. dy: $ log(D12) - log(lagged_Index) $
 3. ep: $ log(E12) - log(Index) $
 4. de: $ log(D12) - log(E12) $
 5. svar
 6. tbl
 7. lty
 8. ltr
 9. tms: $ lty - tbl $
 10. dfy: $ BAA-rated  corporate bond yields - AAA-rated  corporate bond yields $
 11. dfr: $ long-term  corporate bond - long-term  government bond returns $
 12. infl
 

作者根据上述12个变量，本文中构造了如下变量：
 1. exc: lagged returns
 2. ltr
 3. vol: 同前述svar
 4. dp
 5. ep
 6. Rfree
 7. rrel: $ Rfree_t - (Rfree_t + Rfree_{t-1} + Rfree_{t-2})/3$
 8. lty
 9. tms
 10. defsper: 同前述dfy
 11. infl
 12. de

***时间区间：由Welch and Goyal (2008) 的1927\~2005延长到1927\~2008***  
***因变量：S&P 500 index returns including dividends***

## 2.2 变量构造

In [15]:
def construct_variable(start_year, end_year):
    '''
    构造本文分析所需变量，并截取一定时间范围内的数据

    Parameters:
    ----------
    start_year: 开始时间  (int  yyyymm)
    end_year: 结束时间  (int  yyyymm)

    Returns:
    --------
    构造之后的数据  (pd.DataFrame)
    '''
    ret = np.log(raw_data.Index / raw_data.Index.shift(1))  #因变量
    exc = ret.shift(1)  #exc
    ltr = raw_data.ltr  #ltr
    vol = raw_data.svar  #vol
    dp = np.log(raw_data.D12) - np.log(raw_data.Index)
    ep = np.log(raw_data.E12) - np.log(raw_data.Index)
    Rfree = raw_data.Rfree
    rrel = Rfree - (Rfree + Rfree.shift(1) + Rfree.shift(2)) / 3
    lty = raw_data.lty
    tms = lty - raw_data.tbl
    defsper = raw_data.BAA - raw_data.AAA
    infl = raw_data.infl
    de = np.log(raw_data.D12) - np.log(raw_data.E12)
    df = pd.DataFrame({
        "ret": ret, "exc": exc, "ltr": ltr, "vol": vol, "dp": dp, "ep": ep, "Rfree": Rfree, "rrel": rrel, "lty": lty,
        "tms": tms, "defsper": defsper, "infl": infl, "de": de
    })  #创建新数据框
    df = df.loc[start_year:end_year]
    return df

In [16]:
data = construct_variable(192701, 200812)

变量构造后数据如下：

In [17]:
data.head()
#data.to_excel("monthly_data.xlsx")

,ret,exc,ltr,vol,dp,ep,Rfree,rrel,lty,tms,defsper,infl,de
yyyymm,,,,,,,,,,,,,
192701,-0.020975,0.016443,0.0075,0.000470,-2.942374,-2.374773,0.002692,0.000011,0.0351,0.0028,0.0095,-0.011299,-0.567601
192702,0.046589,-0.020975,0.0088,0.000287,-2.979535,-2.430353,0.002742,0.000078,0.0347,0.0018,0.0092,-0.005714,-0.549182
192703,0.006482,0.046589,0.0253,0.000924,-2.976535,-2.445079,0.002667,-0.000033,0.0331,0.0011,0.0092,-0.005747,-0.531456
192704,0.017082,0.006482,-0.0005,0.000603,-2.984225,-2.471309,0.002825,0.000081,0.0333,-0.0006,0.0090,0.000000,-0.512916
192705,0.050905,0.017082,0.0109,0.000392,-3.025963,-2.531446,0.002775,0.000019,0.0327,-0.0006,0.0093,0.005780,-0.494518
